# MCP(Model Context Protocol)

- **MCP(Model Context Protocol)** 는 LLM 애플리케이션이 데이터 소스와 도구에 연결하는 방법을 표준화하는 오픈 프로토콜임

- **USB-C와 같은 개념**으로, 다양한 AI 애플리케이션을 다양한 데이터 소스와 도구에 연결하는 표준화된 방법을 제공함

- **연결의 표준화**를 통해 LLM과 외부 시스템 간의 상호 운용성을 향상시킴

    | 구성요소 | 설명 | 제어 주체 | 예시 |
    |----------|------|-----------|------|
    | **Tools** | LLM이 호출하는 함수 | 모델 | 날씨 조회, 계산기 |
    | **Resources** | 클라이언트가 읽는 데이터 | 애플리케이션 | 설정, 문서 |
    | **Prompts** | 재사용 가능한 프롬프트 템플릿 | 사용자 | 요약, 번역 |


- **MCP 아키텍처**

    ```
    ┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
    │   MCP Host      │     │   MCP Client    │     │   MCP Server    │
    │  (Claude, IDE)  │────▶│  (프로토콜 연결)    │────▶│   (도구/리소스)   │
    └─────────────────┘     └─────────────────┘     └─────────────────┘
    ```

- **장점:**
    - **표준화**: 다양한 LLM 제공업체와 클라이언트 간의 호환성
    - **재사용성**: 한 번 만든 서버를 여러 클라이언트에서 사용 가능
    - **확장성**: 기존 도구와 서비스와의 손쉬운 통합

---

### 1. **개발 환경 설정** 

- **시스템 요구사항**
    - macOS, Linux, Windows 등 다양한 운영체제에서 지원
    - Python 3.10 이상 필요
    - `uv` 패키지 매니저 (권장)

- **uv 설치**

    - **macOS/Linux:**
        ```bash
        curl -LsSf https://astral.sh/uv/install.sh | sh
        ```

    - **Windows:**
        ```powershell
        powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"
        ```
---

### 2. **프로젝트 초기화**

- **새 프로젝트 생성**
    ```bash
    # 새 프로젝트 디렉토리 생성
    uv init my-mcp-server
    cd my-mcp-server
    ```
- **가상 환경 생성 및 활성화**
    ```bash
    uv venv
    source .venv/bin/activate  # Linux/macOS
    # .venv\Scripts\activate    # Windows
    ```
- **필수 패키지 설치**
    ```bash
    # MCP SDK와 HTTP 클라이언트 설치
    uv add "mcp[cli]" httpx
    ```

---

### 3. **MCP의 세 가지 기본 요소**

#### Tools (도구) - 모델 제어
- LLM이 실행할 수 있는 함수
- 외부 시스템과의 상호작용, 계산 수행 등
- **모델 제어**: AI 모델이 자동으로 호출 가능
- 예: 날씨 조회, 데이터베이스 쿼리, API 호출

#### Resources (리소스) - 애플리케이션 제어
- 클라이언트가 읽을 수 있는 데이터와 콘텐츠
- 파일, 데이터베이스, API 응답 등
- **애플리케이션 제어**: 클라이언트가 사용 시점 결정
- 예: 설정 파일, 사용자 데이터, 문서

#### Prompts (프롬프트) - 사용자 제어
- 재사용 가능한 프롬프트 템플릿과 워크플로우
- 표준화된 LLM 상호작용 제공
- **사용자 제어**: 사용자가 명시적으로 선택
- 예: 코드 리뷰 프롬프트, 요약 템플릿

### Transport 옵션

| Transport | 사용 시점 | 장점 | 단점 |
|-----------|-----------|------|------|
| **stdio** | 로컬 프로세스 간 통신 | 간단, 보안 | 같은 머신만 |
| **streamable-http** | 네트워크 통신 | 원격 접근 가능 | 설정 복잡 |

- **[참고] 통신 프로토콜**: JSON-RPC 2.0을 사용하여 메시지를 교환
    - **Requests**: 응답을 기대하는 메시지
    - **Results**: 요청에 대한 성공 응답
    - **Errors**: 요청 실패를 나타내는 응답
    - **Notifications**: 응답을 기대하지 않는 일방향 메시지

---

### 4. **MCP 서버 만들기 (실습)**

`(0) 패키지 설치 및 Import`

> **참고**: Jupyter 노트북에서는 MCP 서버를 직접 실행하기 어렵습니다. 

> 아래 코드는 FastMCP의 구조를 이해하기 위한 예시이며, 실제 서버 실행은 별도 Python 파일에서 수행합니다.

In [ ]:
# MCP 패키지 설치 확인
# 설치: pip install "mcp[cli]" httpx

try:
    from mcp.server.fastmcp import FastMCP
    print("✅ MCP SDK 설치됨")
except ImportError:
    print("❌ MCP SDK가 설치되지 않았습니다.")
    print("설치 명령어: pip install 'mcp[cli]' httpx")
    print("또는: uv add 'mcp[cli]' httpx")

`(1) FastMCP 서버 기본 구조`

FastMCP는 MCP 서버를 쉽게 만들 수 있는 고수준 인터페이스입니다.

In [ ]:
try:
    from mcp.server.fastmcp import FastMCP
    
    # FastMCP 서버 인스턴스 생성
    mcp = FastMCP("Demo Server", json_response=True)
    
    # Tool 정의: LLM이 호출할 수 있는 함수
    @mcp.tool()
    def add(a: int, b: int) -> int:
        """두 숫자를 더합니다"""
        return a + b
    
    @mcp.tool()
    def multiply(a: int, b: int) -> int:
        """두 숫자를 곱합니다"""
        return a * b
    
    @mcp.tool()
    def get_weather(city: str) -> str:
        """도시의 날씨 정보를 반환합니다"""
        weather_data = {
            "서울": "맑음, 15°C",
            "부산": "흐림, 18°C", 
            "제주": "비, 20°C"
        }
        return weather_data.get(city, f"{city}의 날씨 정보를 찾을 수 없습니다")
    
    print("✅ 등록된 도구 (Tools):")
    for tool in mcp._tools:
        print(f"  - {tool}")
        
except ImportError:
    print("MCP SDK를 설치한 후 실행해주세요.")

`(2) Resource 정의`

Resource는 클라이언트가 읽을 수 있는 데이터를 정의합니다. URI 패턴을 사용하여 동적 리소스를 만들 수 있습니다.

In [ ]:
try:
    # Resource 정의: 클라이언트가 읽을 수 있는 데이터
    
    @mcp.resource("greeting://{name}")
    def get_greeting(name: str) -> str:
        """인사말을 반환합니다"""
        return f"안녕하세요, {name}님!"
    
    @mcp.resource("config://settings")
    def get_settings() -> str:
        """설정 정보를 반환합니다"""
        import json
        settings = {
            "theme": "dark",
            "language": "ko",
            "version": "1.0.0"
        }
        return json.dumps(settings, ensure_ascii=False)
    
    print("✅ 등록된 리소스 (Resources):")
    print("  - greeting://{name} : 동적 인사말")
    print("  - config://settings : 설정 정보")
    
except NameError:
    print("먼저 위의 셀을 실행하여 mcp 인스턴스를 생성해주세요.")

`(3) Prompt 정의`

Prompt는 재사용 가능한 프롬프트 템플릿을 정의합니다. 사용자가 명시적으로 선택하여 사용합니다.

In [ ]:
try:
    # Prompt 정의: 재사용 가능한 프롬프트 템플릿
    
    @mcp.prompt()
    def summarize(text: str) -> str:
        """텍스트 요약 프롬프트를 생성합니다"""
        return f"다음 텍스트를 한국어로 3줄로 요약해주세요:\n\n{text}"
    
    @mcp.prompt()
    def code_review(code: str, language: str = "python") -> str:
        """코드 리뷰 프롬프트를 생성합니다"""
        return f"""다음 {language} 코드를 리뷰해주세요:

```{language}
{code}
```

다음 관점에서 분석해주세요:
1. 코드 품질 및 가독성
2. 잠재적 버그 또는 개선점
3. 성능 최적화 제안"""
    
    @mcp.prompt()
    def translate(text: str, target_language: str = "영어") -> str:
        """번역 프롬프트를 생성합니다"""
        return f"다음 텍스트를 {target_language}로 번역해주세요:\n\n{text}"
    
    print("✅ 등록된 프롬프트 (Prompts):")
    print("  - summarize: 텍스트 요약")
    print("  - code_review: 코드 리뷰")
    print("  - translate: 번역")
    
except NameError:
    print("먼저 위의 셀을 실행하여 mcp 인스턴스를 생성해주세요.")

`(4) 서버 실행 코드 (별도 파일)`

아래 코드를 `weather_server.py` 파일로 저장하고 터미널에서 실행합니다.

In [ ]:
%%writefile weather_server.py
# 서버 코드 예시 - 이 코드를 weather_server.py로 저장
# weather_server.py

from mcp.server.fastmcp import FastMCP

# 서버 인스턴스 생성
mcp = FastMCP("Weather Server", json_response=True)

# 날씨 데이터 (실제로는 API 호출)
WEATHER_DATA = {
    "서울": {"temp": 15, "condition": "맑음", "humidity": 45},
    "부산": {"temp": 18, "condition": "흐림", "humidity": 60},
    "제주": {"temp": 20, "condition": "비", "humidity": 80},
}

@mcp.tool()
def get_weather(city: str) -> dict:
    """도시의 날씨 정보를 조회합니다"""
    if city in WEATHER_DATA:
        return WEATHER_DATA[city]
    return {"error": f"{city}의 날씨 정보를 찾을 수 없습니다"}

@mcp.tool()
def get_temperature(city: str) -> str:
    """도시의 온도를 조회합니다"""
    if city in WEATHER_DATA:
        return f"{city}의 현재 온도: {WEATHER_DATA[city]['temp']}°C"
    return f"{city}의 온도 정보를 찾을 수 없습니다"

@mcp.resource("weather://{city}")
def weather_resource(city: str) -> str:
    """날씨 리소스를 반환합니다"""
    import json
    if city in WEATHER_DATA:
        return json.dumps(WEATHER_DATA[city], ensure_ascii=False)
    return json.dumps({"error": "not found"})

@mcp.prompt()
def weather_report(city: str) -> str:
    """날씨 보고서 프롬프트를 생성합니다"""
    return f"""다음 도시의 날씨 정보를 분석하고 오늘의 활동 추천을 해주세요:

도시: {city}

분석 내용:
1. 현재 날씨 상태
2. 실외 활동 적합성
3. 준비물 추천"""

if __name__ == "__main__":
    stdio 모드로 실행 (기본)
    mcp.run()

`(5) MCP Inspector로 테스트`

MCP SDK에는 서버를 테스트할 수 있는 Inspector 도구가 포함되어 있습니다.

```bash
# 서버를 Inspector로 테스트
uv run mcp dev weather_server.py

# 또는 직접 Inspector 실행
npx @modelcontextprotocol/inspector uv run weather_server.py
```

Inspector를 통해:
- 등록된 Tools, Resources, Prompts 확인
- 도구 호출 테스트
- 리소스 읽기 테스트
- 프롬프트 생성 테스트

---

### 5. **클라우드 배포 (Streamable HTTP / SSE)**

로컬 컴퓨터가 아닌 클라우드 서버(Render, Railway 등)에 배포하여 언제 어디서나 접속 가능한 MCP 서버를 만들 수 있습니다. 이를 위해서는 `stdio` 방식 대신 `sse` (Server-Sent Events) 방식을 사용하여 HTTP 통신을 해야 합니다.

- **SSE (Server-Sent Events)**: 서버가 클라이언트로 데이터를 스트리밍할 수 있는 단방향 통신 프로토콜

In [ ]:
%%writefile weather_server_sse.py
# 클라우드 배포용 서버 코드 (SSE 지원)
# weather_server_sse.py

import os
from mcp.server.fastmcp import FastMCP

# 1. 호스트와 포트 지정 (클라우드 환경 대응)
# 클라우드(Render 등)는 PORT 환경변수로 포트를 지정해줍니다.
port = int(os.environ.get("PORT", 8000))
mcp = FastMCP("Weather Server SSE", host="0.0.0.0", port=port)

# ... (기존 Tools/Resources 코드는 단순화를 위해 생략하거나 필요시 추가) ...
# 데모를 위해 기본 데이터만 포함합니다.

WEATHER_DATA = {
    "서울": {"temp": 15, "condition": "맑음", "humidity": 45},
    "부산": {"temp": 18, "condition": "흐림", "humidity": 60},
    "제주": {"temp": 20, "condition": "비", "humidity": 80},
}

@mcp.tool()
def get_weather(city: str) -> dict:
    """도시의 날씨 정보를 조회합니다"""
    return WEATHER_DATA.get(city, {"error": "Not found"})

if __name__ == "__main__":
    # 2. 실행 모드를 'sse'로 설정
    print(f"Starting SSE server on port {port}...")
    mcp.run(transport="sse")

#### **무료 클라우드 배포 가이드 (Render/Railway)**

1.  **필수 파일 준비**
    - `weather_server_sse.py` (위 코드)
    - `requirements.txt`: 의존성 명시
        ```text
        mcp[cli]
        uvicorn
        ```

2.  **GitHub 저장소 업로드**
    - 위 파일들을 포함한 GitHub 리포지토리 생성

3.  **서비스 생성 (Render 예시)**
    - [Render.com](https://render.com) 가입 및 로그인
    - **New +** 버튼 -> **Web Service** 선택
    - GitHub 저장소 연결
    - **설정 입력**:
        - **Runtime**: Python 3
        - **Build Command**: `pip install -r requirements.txt`
        - **Start Command**: `python weather_server_sse.py`
    - **Environment Variables** (중요)
        - `PORT`: `8000` (또는 원하는 포트)
        - *주의: 코드가 `os.environ.get("PORT")`를 사용하므로, 클라우드 설정의 PORT 값에 맞춰 자동으로 실행됩니다.*

4.  **MCP 클라이언트 연결**
    - 배포 후 제공받은 URL (예: `https://my-mcp.onrender.com/sse`)을 클라이언트 설정에 입력

---

### **[기타 참고자료]**
- [MCP 문서](https://modelcontextprotocol.io/introduction)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
- [llms.txt 허브](https://llmstxthub.com/)
- [LangChain Adapters](https://github.com/langchain-ai/langchain-mcp-adapters)